In [1]:
import pandas as pd
import numpy as np

In [2]:
# const_values
travelcard_price = -5000
premium_price = -10000

travel_cashback_limit = 50000
premium_cashback_limit = 100000


In [3]:
client_url = "./client.csv"
url_transaction = "./client_1_transactions_3m.csv"
url_transfer = "./client_1_transfers_3m.csv"

In [4]:
!ls

'bcc (2).ipynb'		        client_info_sm.xlsx	      result.csv
 client_1_transactions_3m.csv   data			      Генерация_.ipynb
 client_1_transfers_3m.csv      Foreign_Exchange_Rates.xlsx
 client.csv		        pipeline.ipynb


In [5]:
client = pd.read_csv(client_url)
transfers_df = pd.read_csv(url_transfer)
transactions_df = pd.read_csv(url_transaction)

# Конвертация валют
  

In [6]:
transactions_df['date'] = pd.to_datetime(transactions_df['date']).dt.strftime('%d.%m.%Y')
transfers_df['date'] = pd.to_datetime(transfers_df['date']).dt.strftime('%d.%m.%Y')

In [7]:
exchange_rate = pd.read_excel("./Foreign_Exchange_Rates.xlsx")
exchange_rate = exchange_rate.set_index('Date')

In [8]:
exchange_rate['KZT'] = 1

In [9]:
transactions_df['converted_amount'] = transactions_df.apply(lambda x: exchange_rate.loc[x['date']][x['currency']] * x['amount'], axis=1)
transfers_df['converted_amount'] = transfers_df.apply(lambda x: exchange_rate.loc[x['date']][x['currency']] * x['amount'], axis=1)

# Рассчет purchases, balance, deposit

In [10]:
transactions_df['date_m'] = transactions_df['date'].str[3:]
transfers_df['date_m'] = transfers_df['date'].str[3:]

In [11]:
purchases = transactions_df.groupby(['client_code', 'date_m'])['converted_amount'].sum()

In [12]:
deposits = transfers_df[transfers_df['type'] == 'deposit_topup_out'].groupby(['client_code', 'date_m'])['converted_amount'].sum()

In [13]:
transfers_df

,client_code,name,product,status,city,date,type,direction,amount,currency,converted_amount,date_m
0,1,Айгерим,Карта для путешествий,зп,Алматы,01.06.2025,card_out,out,9359.56,KZT,9359.56,06.2025
1,1,Айгерим,Карта для путешествий,зп,Алматы,02.06.2025,card_out,out,17590.68,KZT,17590.68,06.2025
2,1,Айгерим,Карта для путешествий,зп,Алматы,02.06.2025,card_out,out,19677.33,KZT,19677.33,06.2025
3,1,Айгерим,Карта для путешествий,зп,Алматы,03.06.2025,card_out,out,37686.28,KZT,37686.28,06.2025
4,1,Айгерим,Карта для путешествий,зп,Алматы,03.06.2025,card_out,out,7880.48,KZT,7880.48,06.2025
...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,Айгерим,Карта для путешествий,зп,Алматы,30.08.2025,card_out,out,36485.00,KZT,36485.00,08.2025
296,1,Айгерим,Карта для путешествий,зп,Алматы,30.08.2025,p2p_out,out,24796.82,KZT,24796.82,08.2025
297,1,Айгерим,Карта для путешествий,зп,Алматы,31.08.2025,p2p_out,out,18330.69,KZT,18330.69,08.2025
298,1,Айгерим,Карта для путешествий,зп,Алматы,31.08.2025,card_out,out,16982.82,KZT,16982.82,08.2025


In [14]:
transfers_df.loc[transfers_df['type'] == 'invest_in', 'converted_amount'] *= -1


In [15]:
invest = transfers_df[transfers_df['type'].isin(['invest_out', 'invest_in'])].groupby(['client_code', 'date_m'])['converted_amount'].sum()

In [16]:
transfers_df.loc[transfers_df['type'] == 'invest_in', 'converted_amount'] *= -1


In [17]:
client_info = client[['client_code', 'avg_monthly_balance_KZT']]

In [18]:
purchases_deposits = pd.merge(purchases, deposits, on=['client_code', 'date_m'], how='outer')
purchases_deposits = pd.merge(purchases_deposits, invest, on=['client_code', 'date_m'], how='outer')
purchases_deposits.reset_index(inplace=True)

In [19]:
client_info = pd.merge(purchases_deposits, client_info, on='client_code', how='outer')
client_info.fillna(0, inplace=True)

In [20]:
client_info.rename(columns={'converted_amount_x': 'purchase', 'converted_amount_y': 'deposit', 'converted_amount': 'invest'}, inplace=True)

In [21]:
client_info

,client_code,date_m,purchase,deposit,invest,avg_monthly_balance_KZT
0,1,06.2025,770614.49,0.0,0.0,92643
1,1,07.2025,1118856.15,0.0,0.0,92643
2,1,08.2025,737443.63,0.0,0.0,92643


# Card costs


In [22]:
client_info["travelcard_cost"] = client_info.apply(
    lambda x: 0 if (x['purchase'] >= 500000
                    or x['avg_monthly_balance_KZT'] > 3000000
                    or x['deposit'] > 3000000)
    else travelcard_price,
    axis=1
)


In [23]:
client_info["premium_cost"] = client_info.apply(
    lambda x: 0 if (x['purchase'] >= 1000000
                    or x['avg_monthly_balance_KZT'] > 15000000
                    or x['deposit'] > 15000000)
    else premium_price,
    axis=1
)


In [24]:
client_info["credit_card"] = 0

# Travel card cashback


In [25]:
travel_cashback_4percent = ['Путешествия', 'Tакси', 'Поезда', 'Самолеты', 'Авиабилеты', 'Отели', 'Аренда авто', 'Авто']

In [26]:
travel_cashback = transactions_df[transactions_df['category'].isin(travel_cashback_4percent)].groupby(['client_code', 'date_m'])['converted_amount'].sum()
travel_cashback *= 0.04
travel_cashback.rename('travel_cashback', inplace=True)


client_code  date_m 
1            06.2025     1584.9456
             07.2025    14795.8860
Name: travel_cashback, dtype: float64

In [27]:
client_info = pd.merge(client_info, travel_cashback, on=['client_code', 'date_m'], how='outer')
client_info.fillna(0, inplace=True)


In [28]:
client_info['travel_cashback'].unique()

array([ 1584.9456, 14795.886 ,     0.    ])

# Premium cashback

In [29]:
vklad = client_info.groupby(['client_code'])['deposit'].sum() + client_info.groupby(['client_code'])['invest'].sum()

In [30]:
percent = np.select(
    [
        vklad >= 6000000,
        vklad >= 1000000
    ],
    [
        0.04,
        0.03
    ],
    default=0.02
)

percent = pd.Series(percent, index=vklad.index, name="percent")

In [31]:
premium_cashback = transactions_df.groupby(['client_code', 'date_m'])['converted_amount'].sum()
premium_cashback = (
    premium_cashback *
    premium_cashback.index.get_level_values(0).map(percent).fillna(0.02)
)
premium_cashback.rename('premium_cashback', inplace=True)


client_code  date_m 
1            06.2025    15412.2898
             07.2025    22377.1230
             08.2025    14748.8726
Name: premium_cashback, dtype: float64

In [32]:
client_info = pd.merge(client_info, premium_cashback, on=['client_code', 'date_m'], how='outer')
client_info.fillna(0, inplace=True)


# Credit card cashback

In [33]:
credit_card_cashback = (
    transactions_df
    .groupby(['client_code', 'date_m'])
    .apply(lambda g: g[['category', 'converted_amount']]
                   .groupby('category')['converted_amount']
                   .sum()
                   .nlargest(3)
                   .sum())
)
credit_card_cashback.rename('credit_card_cashback', inplace=True)

client_code  date_m 
1            06.2025    474228.12
             07.2025    650310.66
             08.2025    437614.47
Name: credit_card_cashback, dtype: float64

In [34]:
credit_card_cashback *= 0.1

In [35]:
client_info = pd.merge(client_info, credit_card_cashback, on=['client_code', 'date_m'], how='outer')
client_info.fillna(0, inplace=True)


In [36]:
client_info['travel_cashback'] = client_info['travel_cashback'].apply(lambda x: min(x, travel_cashback_limit))
client_info['premium_cashback'] = client_info['premium_cashback'].apply(lambda x: min(x, premium_cashback_limit))

In [37]:
client_info

,client_code,date_m,purchase,deposit,invest,avg_monthly_balance_KZT,travelcard_cost,premium_cost,credit_card,travel_cashback,premium_cashback,credit_card_cashback
0,1,06.2025,770614.49,0.0,0.0,92643,0,-10000,0,1584.9456,15412.2898,47422.812
1,1,07.2025,1118856.15,0.0,0.0,92643,0,0,0,14795.8860,22377.1230,65031.066
2,1,08.2025,737443.63,0.0,0.0,92643,0,-10000,0,0.0000,14748.8726,43761.447


# Transfers


In [38]:
transfers_df['type'].unique()

array(['card_out', 'p2p_out', 'atm_withdrawal', 'utilities_out',
       'cashback_in', 'loan_payment_out', 'card_in', 'salary_in',
       'refund_in'], dtype=object)

In [39]:
import pandas as pd

invest_ops = ['invest_out', 'invest_in']
fx_ops = ['fx_buy', 'fx_sell']
gold_ops = ['gold_buy_out', 'gold_sell_in']
credit_ops = ['loan_payment_out', 'cc_repayment_out', 'installment_payment_out']

agg_df = (
    transfers_df
    .groupby(['client_code', 'date_m'])
    .apply(lambda g: pd.Series({
        'fx_count': g[g['type'].isin(fx_ops)].shape[0],
    }))
)


In [40]:
client_info = pd.merge(client_info, agg_df, on=['client_code', 'date_m'], how='outer')
client_info.fillna(0, inplace=True)


# Logic

In [41]:
client_info['premium_cashback'] = client_info['premium_cost'] + client_info['premium_cashback']
client_info['travel_cashback'] = client_info['travelcard_cost'] + client_info['travel_cashback']

client_info.drop(columns=['travelcard_cost', 'premium_cost', 'credit_card'], inplace=True)

In [42]:
client_info_sm = client_info.drop(columns=['date_m']).groupby('client_code').sum()

In [43]:
vip = ['Золотые слитки', 'Инвестиции', 'Депозит Накопительный', 'card']
valut = ['Обмен валют', 'card']
middle = ['Инвестиции', 'Депозит Сберегательный', 'card']
standart = ['Кредит наличными', 'Депозит Мультивалютный', 'card']

In [44]:
client_info_sm = pd.merge(client_info_sm, client[['client_code', 'status']], on=['client_code'], how='left')


In [45]:
client_info_sm['credit_card_cashback'] = client_info_sm['credit_card_cashback'].apply(lambda x: min(x, 240000))

In [46]:

cashback_cols = ['travel_cashback', 'premium_cashback', 'credit_card_cashback']

client_info_sm['cashback'] = client_info_sm[cashback_cols].max(axis=1)

client_info_sm['card'] = client_info_sm[cashback_cols].idxmax(axis=1)

name_map = {
    'travel_cashback': 'Карта для путешествий',
    'premium_cashback': 'Премиальная карта',
    'credit_card_cashback': 'Кредитная карта'
}

client_info_sm['card'] = client_info_sm['card'].map(name_map)


In [47]:
client_info_sm.drop(columns=['travel_cashback', 'premium_cashback', 'credit_card_cashback'], inplace=True)

In [48]:
client_info_sm['fx_count'].value_counts()

0    1
Name: fx_count, dtype: int64

In [49]:
import random
def choose_product(row):
    if (row['status'] == "Премиальный клиент") or (row['avg_monthly_balance_KZT'] >= 1_000_000*3):
        product = random.choice(vip)
        if product.endswith("card"):
            return random.choice(['Премиальная карта'])
        return product

    elif row['fx_count'] >= 12 * 3:
        product = random.choice(valut)
        if product.endswith("card"):
            return row['card']
        return product

    elif row['avg_monthly_balance_KZT'] >= 100_000 * 3:
        product = random.choice(middle)
        if product.endswith("card"):
            return row['card']
        return product

    else:
        product = random.choice(standart)
        if product.endswith("card"):
            return row['card']
        return product

client_info_sm['recommended_product'] = client_info_sm.apply(choose_product, axis=1)


In [50]:
client_info_sm = pd.merge(client[['client_code', 'name', 'age']], client_info_sm, on=['client_code'], how='left')


In [51]:
client_info_sm

,client_code,name,age,purchase,deposit,invest,avg_monthly_balance_KZT,fx_count,status,cashback,card,recommended_product
0,1,Айгерим,29,2626914.27,0.0,0.0,277929,0,Зарплатный клиент,156215.325,Кредитная карта,Депозит Мультивалютный


# Генерация текста

In [52]:
templates = {
    "Карта для путешествий": {
        "<30": "{name}, за последние 3 месяца вы могли заработать {cashback} ₸ на поездках и такси 🚕. Тревел-карта вернёт часть расходов и даст привилегии Visa Signature. Оформить?",
        "30+": "{name}, за последние 3 месяца ваши поездки и такси принесли бы {cashback} ₸. С тревел-картой вы вернёте ещё больше и пользуетесь бонусами Visa Signature. Посмотреть карту."
    },
    "Премиальная карта": {
        "<30": "{name}, за 3 месяца ваш потенциальный кешбэк — {cashback} ₸. Премиальная карта даёт до 4% на рестораны и ювелирку ✨, плюс снятие наличных без заморочек. Посмотреть карту?",
        "30+": "{name}, за последние 3 месяца ваш кешбэк мог составить {cashback} ₸. Премиальная карта повышает доход и упрощает снятие/переводы наличных. Открыть карту."
    },
    "Кредитная карта": {
        "<30": "{name}, за 3 месяца можно было заработать {cashback} ₸ 😎. До 10% кешбэка и онлайн-сервисы, плюс рассрочка без переплат. Оформить карту?",
        "30+": "{name}, за последние 3 месяца ваш потенциальный кешбэк — {cashback} ₸. Кредитная карта возвращает до 10% на топ-категории и онлайн-услуги, плюс рассрочка без переплат. Посмотреть карту."
    },
    "Обмен валют": {
        "<30": "{name}, часто меняете валюту? 💸 В приложении выгодный курс, без комиссии, и авто-покупка по целевому курсу. Настроить обмен?",
        "30+": "{name}, вы часто меняете валюту. В приложении выгодный курс без комиссии и возможность авто-покупки по целевому курсу. Посмотреть обмен."
    },
    "Кредит наличными": {
        "<30": "{name}, нужен быстрый запас на крупные расходы? 💡 Кредит наличными без залога и справок, онлайн или в отделении, с гибкими выплатами. Узнать лимит?",
        "30+": "{name}, если нужен быстрый кредит на любые цели — оформите кредит наличными онлайн или в отделении. Гибкие выплаты, без залога и справок. Узнать лимит."
    },
    "Депозит Мультивалютный": {
        "<30": "{name}, держите свободные средства? 💰 Мультивалютный депозит 14,50% с доступом к деньгам и пополнением без ограничений. Открыть депозит?",
        "30+": "{name}, у вас остаются свободные средства? Мультивалютный депозит 14,50% с доступом к деньгам и пополнением без ограничений. Посмотреть условия."
    },
    "Депозит Сберегательный": {
        "<30": "{name}, хотите максимальный доход? 🔒 Сберегательный депозит 16,50% до конца срока — храните средства без снятия. Открыть вклад?",
        "30+": "{name}, чтобы получить максимальный доход, разместите средства на сберегательном депозите 16,50% до конца срока. Посмотреть вклад."
    },
    "Депозит Накопительный": {
        "<30": "{name}, планомерно копите? 💡 Накопительный депозит 15,50% с возможностью пополнения — удобно откладывать. Открыть вклад?",
        "30+": "{name}, если хотите копить с доходом 15,50%, накопительный депозит с пополнением — отличный вариант. Посмотреть условия."
    },
    "Инвестиции": {
        "<30": "{name}, хотите начать инвестировать с малых сумм? 📈 Без комиссий первый год, порог от 6 ₸. Попробовать инвестиции?",
        "30+": "{name}, чтобы начать инвестировать с низким порогом и без комиссий первый год, откройте инвестиционный счёт. Посмотреть условия."
    },
    "Золотые слитки": {
        "<30": "{name}, диверсифицируете сбережения? 🪙 Купите золотые слитки 999,9 пробы в приложении или отделении, храните в сейфах. Посмотреть варианты?",
        "30+": "{name}, чтобы сохранить стоимость средств и диверсифицировать портфель, купите золотые слитки 999,9 пробы и храните в сейфах банка. Посмотреть."
    },
}

def generate_push(user):
    age_group = "<30" if user["age"] < 30 else "30+"
    product = user["recommended_product"]
    template = templates.get(product, {}).get(age_group)

    user_data = {
        "name": user["name"],
        "cashback":  f"{int(round(user['cashback'])):,}".replace(",", " ")
    }

    if template:
        return template.format(**user_data)
    else:
        return f"{user['name']}, нет шаблона для выбранного продукта."




In [53]:
client_info_sm["push_notification"] = client_info_sm.apply(generate_push, axis=1)


In [54]:
client_info_sm.rename(columns={'recommended_product': 'product'}, inplace=True)

In [55]:
result = client_info_sm[['client_code', 'product', 'push_notification']]

In [56]:
result

,client_code,product,push_notification
0,1,Депозит Мультивалютный,"Айгерим, держите свободные средства? 💰 Мультив..."
